### Imports

In [21]:
%pip install -r dependencies.txt

  Using cached aiofiles-22.1.0-py3-none-any.whl (14 kB)
  Using cached aiosqlite-0.18.0-py3-none-any.whl (15 kB)
  Using cached anyio-3.6.2-py3-none-any.whl (80 kB)
  Using cached argon2_cffi-21.3.0-py3-none-any.whl (14 kB)
  Using cached argon2_cffi_bindings-21.2.0-cp38-abi3-macosx_10_9_universal2.whl (53 kB)
  Using cached arrow-1.2.3-py3-none-any.whl (66 kB)
  Using cached attrs-22.2.0-py3-none-any.whl (60 kB)
  Using cached Babel-2.12.1-py3-none-any.whl (10.1 MB)
  Using cached bardapi-0.1.38-py3-none-any.whl.metadata (21 kB)
  Using cached beautifulsoup4-4.11.2-py3-none-any.whl (129 kB)
  Using cached bleach-6.0.0-py3-none-any.whl (162 kB)
  Using cached browser_cookie3-0.19.1-py3-none-any.whl.metadata (632 bytes)
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)
  Using cached cffi-1.15.1-cp311-cp311-macosx_11_0_arm64.whl (174 kB)
  Using cached charset_normalizer-3.1.0-cp311-cp311-macosx_11_0_arm64.whl (121 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)


In [22]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import requests
import scipy
import importlib
import helper, consts
importlib.reload(consts)
importlib.reload(helper)

<module 'helper' from '/Users/hoangchu/Downloads/Clinic/Clinic/code/helper.py'>

### CONSTANTS

In [23]:
# USER
USER = "HOANG"
FILE_PATH = consts.PATH_MAP[USER]

ROW = consts.ROW
COL = consts.COL
CLEANED_DATA_PATH = consts.CLEANED_DATA_PATH
DATA_PATH = consts.RAW_DATA_PATH
RESPONSE_NAME = consts.RESPONSE_NAME

TRAIN_START_DATE = "20150101"
TRAIN_END_DATE = "20150601" # Up to but not including
TEST_START_DATE = "20150701"
TEST_END_DATE = "20150801"

In [24]:
REGRESSION_TYPES = helper.Regression('OLS').list_all_regression_types()
REGRESSION_TYPES

1: OLS
2: LASSO
3: XGBOOST


### Read data

In [25]:
train_start, train_end  = helper.get_train_from_testday(TEST_START_DATE)
train_start, train_end
list_of_interacting_terms = [["relvol_nt_0","rrirpnxm_nt_0" ], 
                             ["relvol_lst15_0","rrirpnxm_lst15_0" ],
                             ["relvol_lsthrx15_0","rrirpnxm_lsthrx15_0" ],
                             ["relvol_toxhr_0","rrirpnxm_toxhr_0" ] ]

In [26]:
# x_cols = ["rrirpnxm_nt_0", "rrirpnxm_lst15_0","rrirpnxm_lsthrx15_0", "rrirpnxm_toxhr_0", "relvol_nt_0"]
# train_df = helper.get_df(train_start, train_end, x_cols, FILE_PATH)

train_df = pd.read_csv(consts.CLEANED_DATA_PATH)
pred_cols = list(train_df.columns)
pred_cols.remove(consts.RESPONSE_NAME)



### Interaction Terms

### Transform the data if needed

#### Ordinary Least Squares

In [27]:
importlib.reload(helper)

ols_regression_model = helper.Regression('OLS')
weights = helper.get_weights(train_df)
model_attributes = ols_regression_model.train(train_df, RESPONSE_NAME, weights)
pd.DataFrame({"ols_coefficients": model_attributes}).to_csv("ols_coef", index_label=False)

In [28]:
importlib.reload(helper)
ols_regression_model.evaluate_model(TEST_START_DATE, TEST_END_DATE, pred_cols,FILE_PATH, list_of_interacting_terms )

filenames = ['data.20150701_1200', 'data.20150702_1200', 'data.20150706_1200', 'data.20150707_1200', 'data.20150708_1200', 'data.20150709_1200', 'data.20150710_1200', 'data.20150713_1200', 'data.20150714_1200', 'data.20150715_1200', 'data.20150716_1200', 'data.20150717_1200', 'data.20150720_1200', 'data.20150721_1200', 'data.20150722_1200', 'data.20150723_1200', 'data.20150724_1200', 'data.20150727_1200', 'data.20150728_1200', 'data.20150729_1200', 'data.20150730_1200', 'data.20150731_1200']
Average Weighted Correlation: 0.9046434873984315
Average Mean Return: 0.0065842178796169685
Average weighted Scale Factor: 0.9068187187192163


(0.9046434873984315, 0.0065842178796169685, 0.9068187187192163)

#### OLS w/ Interacting Terms

#### LASSO

In [29]:
lasso_model = helper.Regression('LASSO')
weights = helper.get_weights(train_df)

In [30]:
importlib.reload(helper)
lasso_attributes = lasso_model.train(train_df, RESPONSE_NAME, weights)
pd.DataFrame({"lasso_coefficients": model_attributes}).to_csv("lasso_coef", index_label=False)
lasso_model.evaluate_model(TEST_START_DATE, TEST_END_DATE, pred_cols, FILE_PATH, list_of_interacting_terms)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.0402532044671027e-05, tolerance: 1.927929494361376e-05
  model = cd_fast.enet_coordinate_descent_gram(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.3570905495133455e-05, tolerance: 1.9427250729914277e-05
  model = cd_fast.enet_coordinate_descent_gram(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.590690433232958e-05, tolerance: 1.9178957557132583e-05
  m

filenames = ['data.20150701_1200', 'data.20150702_1200', 'data.20150706_1200', 'data.20150707_1200', 'data.20150708_1200', 'data.20150709_1200', 'data.20150710_1200', 'data.20150713_1200', 'data.20150714_1200', 'data.20150715_1200', 'data.20150716_1200', 'data.20150717_1200', 'data.20150720_1200', 'data.20150721_1200', 'data.20150722_1200', 'data.20150723_1200', 'data.20150724_1200', 'data.20150727_1200', 'data.20150728_1200', 'data.20150729_1200', 'data.20150730_1200', 'data.20150731_1200']
Average Weighted Correlation: 0.8546711224960641
Average Mean Return: 0.006153421506920837
Average weighted Scale Factor: 1.0497856975115367


(0.8546711224960641, 0.006153421506920837, 1.0497856975115367)

### XGBoost



In [31]:
xgb_model = helper.Regression('XGboost')
weights = helper.get_weights(train_df)
xgb_attributes = xgb_model.train(train_df, RESPONSE_NAME, weights)
xgb_model.evaluate_model(TEST_START_DATE, TEST_END_DATE, pred_cols, FILE_PATH, list_of_interacting_terms)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/core.py:726: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)


filenames = ['data.20150701_1200', 'data.20150702_1200', 'data.20150706_1200', 'data.20150707_1200', 'data.20150708_1200', 'data.20150709_1200', 'data.20150710_1200', 'data.20150713_1200', 'data.20150714_1200', 'data.20150715_1200', 'data.20150716_1200', 'data.20150717_1200', 'data.20150720_1200', 'data.20150721_1200', 'data.20150722_1200', 'data.20150723_1200', 'data.20150724_1200', 'data.20150727_1200', 'data.20150728_1200', 'data.20150729_1200', 'data.20150730_1200', 'data.20150731_1200']
Average Weighted Correlation: 0.9027810391350344
Average Mean Return: 0.006552297860255734
Average weighted Scale Factor: 0.9356580972671509


(0.9027810391350344, 0.006552297860255734, 0.9356581)

In [32]:
importances = xgb_model.model.feature_importances_
feature_names = train_df.columns  # Replace with your feature names

# Display feature importance scores
for feature, importance in zip(feature_names, importances):
    print(f"{feature}: {importance}")

wt: 0.3401058614253998
today: 0.8090566396713257
tonight: 0.8442369699478149
tmwam: 0.26880770921707153
dn3sttmwmd: -0.827448844909668
dn1: -0.24777624011039734
dn4x1: -0.00016346467600669712
cftorrrelstd_open_0: -6.049116564099677e-05
liqlog_open_0: 0.05540808290243149
llirpnxm_am_1: -0.0028804249595850706
llirpnxm_dy_12to16: 0.015000131912529469
llirpnxm_dy_17to21: 0.03327547013759613
llirpnxm_dy_1to3: -0.02553272433578968
llirpnxm_dy_4to6: 0.02797333337366581
llirpnxm_dy_7to11: -0.5245780348777771
llirpnxm_lst15_0: 0.039608150720596313
llirpnxm_lsthrx15_0: 0.09408893436193466
llirpnxm_md_1: -0.03809340298175812
llirpnxm_nt_0: -0.004170230124145746
llirpnxm_nt_1: 0.0004319337022025138
llirpnxm_nt_12to16: 0.0033193244598805904
llirpnxm_nt_17to21: 0.024361031129956245
llirpnxm_nt_1to3: -0.026801757514476776
llirpnxm_nt_4to6: -0.008152020163834095
llirpnxm_nt_7to11: -0.04327008128166199
llirpnxm_toxhr_0: 0.00015101079770829529
mocrelvol_open_1: 1.7682967154541984e-05
mocrelvol_open_13to